In [8]:
import pandas as pd
import plotly.express as px
import statsmodels.formula.api as smf
import numpy as np

Lectura de los datos desde un archivo csv

In [2]:
df = pd.read_csv("data/glassdoordata.csv")
df.head()

,jobtitle,gender,age,performance,education,department,seniority,income,bonus
0,Graphic Designer,Female,18,5,College,Operations,2,42363,9938
1,Software Engineer,Male,21,5,College,Management,5,108476,11128
2,Warehouse Associate,Female,19,4,PhD,Administration,5,90208,9268
3,Software Engineer,Male,20,5,Masters,Sales,4,108080,10154
4,Graphic Designer,Male,26,5,Masters,Engineering,5,99464,9319


# Análisis descriptivo: Distribuciones de probabilidad de los campos

In [3]:
df.describe()

,age,performance,seniority,income,bonus
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,41.393000,3.037000,2.971000,94472.653000,6467.161000
std,14.294856,1.423959,1.395029,25337.493272,2004.377365
min,18.000000,1.000000,1.000000,34208.000000,1703.000000
25%,29.000000,2.000000,2.000000,76850.250000,4849.500000
50%,41.000000,3.000000,3.000000,93327.500000,6507.000000
75%,54.250000,4.000000,4.000000,111558.000000,8026.000000
max,65.000000,5.000000,5.000000,179726.000000,11293.000000


In [5]:
numeric_columns = ['age', 'performance', 'seniority', 'income', 'bonus']
for i_col in numeric_columns:
    fig = px.histogram(df, x=i_col)
    fig.show()

Conclusiones:
 - La edad, el rendimiento y la seniority tienen una distribución de datos constante, lo cual se puede originar por el tipo de muestreo utilizado.
 - El income y el bonus tienen una distribución de probabilidad en forma de gausiana.

# Análisis descriptivo: Valores atípicos, nulos, o vacios

In [9]:
df_columns = list(df.columns)
null_values = ["",np.nan, None]
for col_name in df_columns:
    n_points = len(df[(df[col_name].isin(null_values)) | (df[col_name].isnull())])
    print(f"En la columna {col_name} se encuentran {n_points} filas vacias")

En la columna jobtitle se encuentran 0 filas vacias
En la columna gender se encuentran 0 filas vacias
En la columna age se encuentran 0 filas vacias
En la columna performance se encuentran 0 filas vacias
En la columna education se encuentran 0 filas vacias
En la columna department se encuentran 0 filas vacias
En la columna seniority se encuentran 0 filas vacias
En la columna income se encuentran 0 filas vacias
En la columna bonus se encuentran 0 filas vacias


conclusiones:
- Las columnas no presentan datos vacios
- En las distribuciones de probabilidad no se observan outliers
- No existen columnas relacionadas con leyes economicas o fisicas para revisar inliers y coherencia en los datos

# Análisis descriptivo: Comportamiento de las variables en las diferentes dimensiones

In [11]:
categorical_columns = ['jobtitle','gender','age','performance','education','department','seniority']
for col_name in categorical_columns:
    fig=px.box(df, y="income", title= f"Distribucion de ingresos por {col_name}", color=col_name)
    fig.show()

In [13]:
for col_name in categorical_columns:
    fig=px.box(df, y="bonus", title= f"Distribucion de bonos por {col_name}", color=col_name)
    fig.show()

- top 10 jobtitles según el ingreso total

In [13]:
df["total_income"] = df["income"]+ df["bonus"]
df_agg = df.groupby("jobtitle").agg({"total_income": ["sum", "count"]}).reset_index()
df_agg["avg_income"] = df_agg[("total_income", "sum")] / df_agg[("total_income", "count")]
df_agg = df_agg.sort_values("avg_income", ascending = False)
df_agg.head(10)

jobtitle total_income           avg_income
                                sum count               
5              Manager     11825296    90  131392.177778
8    Software Engineer     12225594   109  112161.412844
2    Financial Analyst     10850188   107  101403.626168
7      Sales Associate      9369054    94   99670.787234
0       Data Scientist     10584270   107   98918.411215
3     Graphic Designer      9572104    98   97674.530612
4                   IT      9314918    96   97030.395833
9  Warehouse Associate      8711860    90   96798.444444
1               Driver      8715879    91   95778.890110
6  Marketing Associate      9770651   118   82802.127119

- Nivel educativo con mejor ingresos

In [14]:
df_agg = df.groupby("education").agg({"total_income": ["sum", "count"]}).reset_index()
df_agg["avg_income"] = df_agg[("total_income", "sum")] / df_agg[("total_income", "count")]
df_agg = df_agg.sort_values("avg_income", ascending = False)
df_agg.head(10)

education total_income           avg_income
                        sum count               
3          PhD     25379835   238  106637.962185
2      Masters     26638910   256  104058.242188
0      College     23780141   241   98672.784232
1  High School     25140928   265   94871.426415

- Media y desviación del rendimiento por cargo

In [22]:
df_describe = df.groupby("jobtitle").describe()
df_describe["performance"]

,count,mean,std,min,25%,50%,75%,max
jobtitle,,,,,,,,
Data Scientist,107.0,2.971963,1.423906,1.0,2.0,3.0,4.00,5.0
Driver,91.0,3.065934,1.340002,1.0,2.0,3.0,4.00,5.0
Financial Analyst,107.0,3.271028,1.418011,1.0,2.0,4.0,4.00,5.0
Graphic Designer,98.0,3.122449,1.466220,1.0,2.0,3.0,4.00,5.0
IT,96.0,3.125000,1.481464,1.0,2.0,3.0,4.25,5.0
Manager,90.0,2.700000,1.302116,1.0,2.0,3.0,4.00,5.0
Marketing Associate,118.0,2.923729,1.409108,1.0,2.0,3.0,4.00,5.0
Sales Associate,94.0,2.797872,1.387965,1.0,2.0,3.0,4.00,5.0
Software Engineer,109.0,3.128440,1.515970,1.0,2.0,3.0,5.00,5.0


- Departamento con personal de mayor antiguedad

In [26]:
df_agg = df.groupby("department").agg({"seniority": "mean"}).reset_index()
df_agg.sort_values("seniority", ascending = False)

,department,seniority
1,Engineering,3.140625
0,Administration,3.103627
2,Management,2.974747
4,Sales,2.879227
3,Operations,2.780952


conclusiones
- La posición de manager presenta una distribución con mayores valores en el ingreso, seguido de ingenieros de software.
- El role con menores ingresos es el de marketing associate, seguido por el role de conductor.
- La media de ingresos por hombres es ligeramente superior en comparación con la de mujeres.
- La distribución del income dado un performance se mantiene constante.
- Los niveles educativos de phD y Masters tienen un mayor ingerso en comparación con college y high school.
- Los departamentos de ventas, ingenieria y gerencia tienen las distribuciones de ingresos más elevadas.
- El ingreso es proporcional al nivel de seniority.
- El área de warehouse associate tiene la mayor distribución en bonos, seguido de software engineer, diseñadores gráficos e IT
- La distribución del bono dado el genero se mantiene constante.
- El bono es proporcional al nivel de rendimiento
- La distribución del bono es ligeramente inferior cuando el nivel de educación es high school
- El departamento de ingenieria tiene una distribución de bonos más elevada
- La distribución del bono es proporcional a la seniority


# Análisis descriptivo: Correlaciones

In [37]:
df.corr()

,age,performance,seniority,income,bonus,total_income
age,1.000000,-0.056875,-0.021414,0.562681,-0.414337,0.533715
performance,-0.056875,1.000000,-0.021127,-0.053622,0.855500,0.014155
seniority,-0.021414,-0.021127,1.000000,0.511096,0.194982,0.530307
income,0.562681,-0.053622,0.511096,1.000000,-0.129481,0.996874
bonus,-0.414337,0.855500,0.194982,-0.129481,1.000000,-0.050736
total_income,0.533715,0.014155,0.530307,0.996874,-0.050736,1.000000


El ingreso total tiene una alta correlación con el nivel de seniority y la edad

# Análisis predictivo: Predicción de ingresos e importancia de features

En progreso